<a href="https://colab.research.google.com/github/mprksa/Blocks2/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mprksa/Blocks2.git

Cloning into 'Blocks2'...
remote: Enumerating objects: 862, done.
remote: Counting objects: 100% (862/862), done.
remote: Compressing objects: 100% (848/848), done.
remote: Total 862 (delta 27), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (862/862), 1.82 GiB | 24.15 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Updating files: 100% (725/725), done.


# **Bagian 1: Import Library dan Definisikan Fungsi**

In [23]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, classification_report
from keras.preprocessing.image import ImageDataGenerator

In [34]:
def load_images_and_labels(image_folder, label_file):
    # Load labels.json
    with open(label_file, 'r') as f:
        label_data = json.load(f)

    # Create class indices for flow_from_directory
    class_indices = {item['id']: item['file_name'] for item in label_data['images']}

    # Load images from the specified folder using OpenCV
    images = []
    labels = []
    for item_id, file_name in class_indices.items():
        image_path = os.path.join(image_folder, file_name)
        image = cv2.imread(image_path)
        if image is not None:
            images.append(image)
            labels.append(item_id)  # Assuming item_id corresponds to labels

    return np.array(images), np.array(labels)


In [36]:
def apply_convolution(image, kernel):
    return cv2.filter2D(image, -1, kernel)

In [37]:
def rgb_to_hsv(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    return hsv_image

In [38]:
def extract_hsv_features_with_convolution(images, labels):
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])

    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])

    features = []
    for image, label in zip(images, labels):
        convolved_image_x = apply_convolution(image, sobel_x)
        convolved_image_y = apply_convolution(image, sobel_y)

        convolved_image = cv2.addWeighted(convolved_image_x, 0.5, convolved_image_y, 0.5, 0)

        hsv_image = rgb_to_hsv(convolved_image)

        hsv_values = hsv_image.reshape(-1, 3)
        features.append((hsv_values, label))
    return features

In [39]:
def prepare_data(features):
    data = []
    labels = []
    for hsv_values, label in features:
        data.append(hsv_values)
        labels.extend([label] * len(hsv_values))
    return np.vstack(data), np.array(labels)


In [40]:
def test_model(knn, test_features, k=3):
    true_labels = []
    predicted_labels = []

    for hsv_values, true_label in test_features:
        ret, results, neighbours, dist = knn.findNearest(hsv_values.astype(np.float32), k=k)
        predicted_label = np.bincount(results.flatten().astype(int)).argmax()

        true_labels.append(true_label)
        predicted_labels.append(predicted_label)

    accuracy = np.mean(np.array(true_labels) == np.array(predicted_labels))
    return accuracy, true_labels, predicted_labels

In [41]:
# Fungsi untuk menghitung nilai rata-rata RGB untuk setiap kelas
def calculate_average_rgb(images, labels):
    class_rgb_values = {}
    for image, label in zip(images, labels):
        if label not in class_rgb_values:
            class_rgb_values[label] = []
        class_rgb_values[label].append(np.mean(image, axis=(0, 1)))

    average_rgb_values = {label: np.mean(np.array(rgb_values), axis=0) for label, rgb_values in class_rgb_values.items()}
    return average_rgb_values

In [42]:
# Fungsi untuk memuat gambar dalam batch
def create_data_generator(image_folder, label_file, target_size=(256, 256), batch_size=32, class_mode='categorical'):
    with open(label_file, 'r') as f:
        label_data = json.load(f)

    class_indices = {item['id']: item['file_name'] for item in label_data['images']}

    datagen = ImageDataGenerator(rescale=1./255)

    generator = datagen.flow_from_directory(
        directory=image_folder,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        classes=list(class_indices.values())
    )

    return generator

# **Bagian 2: Memuat Data dan Mengekstraksi Fitur**

In [ ]:
train_data = load_images_and_labels('Blocks2/train/images', 'Blocks2/train/labels.json')
train_images, train_labels = list(train_data.values())[:2]

test_data = load_images_and_labels('Blocks2/test/images', 'Blocks2/test/labels.json')
test_images, test_labels = list(test_data.values())[:2]

validation_data = load_images_and_labels('Blocks2/validation/images', 'Blocks2/validation/labels.json')
validation_images, validation_labels = list(validation_data.values())[:2]

In [33]:
# Ekstraksi fitur dari citra dengan konvolusi
train_features = extract_hsv_features_with_convolution(train_images, train_labels)
test_features = extract_hsv_features_with_convolution(test_images, test_labels)
validation_features = extract_hsv_features_with_convolution(validation_images, validation_labels)

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'filter2D'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [17]:
# Menampilkan jumlah gambar di setiap folder
print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")
print(f"Number of test images: {len(test_images)}")

Number of training images: 21
Number of validation images: 21
Number of test images: 21


# **Bagian 3: Melatih Model dan Menampilkan Grafik Akurasi dan F1 Score**

In [18]:
# Menyiapkan data untuk pelatihan
train_data, train_labels = prepare_data(train_features)
val_data, val_labels = prepare_data(val_features)

NameError: name 'train_features' is not defined

In [ ]:
# Mencari akurasi untuk berbagai nilai K
k_values = range(1, 11)
train_accuracies = []
val_accuracies = []
val_f1_scores = []

In [ ]:
for k in k_values:
    knn = cv2.ml.KNearest_create()
    knn.train(train_data.astype(np.float32), cv2.ml.ROW_SAMPLE, train_labels.astype(np.int32))

    # Menghitung akurasi pada data pelatihan
    train_accuracy, _, _ = test_model(knn, train_features, k=k)
    train_accuracies.append(train_accuracy)

    # Menghitung akurasi dan F1 Score pada data validasi
    val_accuracy, true_labels, predicted_labels = test_model(knn, val_features, k=k)
    val_accuracies.append(val_accuracy)
    val_f1_scores.append(f1_score(true_labels, predicted_labels, average='weighted'))

In [ ]:
# Menampilkan grafik akurasi dan F1 Score
plt.figure(figsize=(10, 5))
plt.plot(k_values, train_accuracies, label='Training Accuracy', marker='o')
plt.plot(k_values, val_accuracies, label='Validation Accuracy', marker='o')
plt.plot(k_values, val_f1_scores, label='Validation F1 Score', marker='o')
plt.xlabel('Number of Neighbors (K)')
plt.ylabel('Score')
plt.title('KNN Performance for Different K Values')
plt.legend()
plt.grid()
plt.show()

# **Bagian 4: Menghitung Nilai Rata-Rata RGB dan Menampilkan Hasil Uji**

In [ ]:
# Menghitung nilai rata-rata RGB untuk setiap kelas
average_rgb_values = calculate_average_rgb(train_images, train_labels)
print(f"Average RGB values for each class: {average_rgb_values}")

In [ ]:
# Menghitung F1 Score pada data uji
best_k = k_values[np.argmax(val_f1_scores)]
knn = cv2.ml.KNearest_create()
knn.train(train_data.astype(np.float32), cv2.ml.ROW_SAMPLE, train_labels.astype(np.int32))
test_accuracy, test_true_labels, test_predicted_labels = test_model(knn, test_features, k=best_k)
test_f1_score = f1_score(test_true_labels, test_predicted_labels, average='weighted')
print(f"Test Accuracy: {test_accuracy}")
print(f"Test F1 Score: {test_f1_score}")
print(classification_report(test_true_labels, test_predicted_labels))

# **Bagian 5: Menyimpan dan Memuat Model**

In [ ]:
# Menyimpan model ke file
model_filename = 'knn_model.yml'
knn.save(model_filename)
print(f"Model saved to {model_filename}")

In [ ]:
# Memuat model dari file (contoh penggunaan)
knn_loaded = cv2.ml.KNearest_create()
knn_loaded = knn_loaded.load(model_filename)

In [ ]:
# Verifikasi model yang dimuat menghasilkan hasil yang sama
loaded_test_accuracy, loaded_test_true_labels, loaded_test_predicted_labels = test_model(knn_loaded, test_features, k=best_k)
loaded_test_f1_score = f1_score(loaded_test_true_labels, loaded_test_predicted_labels, average='weighted')
print(f"Loaded Test Accuracy: {loaded_test_accuracy}")
print(f"Loaded Test F1 Score: {loaded_test_f1_score}")
print(classification_report(loaded_test_true_labels, loaded_test_predicted_labels))